# Improving Bounds on Elliptic Curve Hidden Number Problem for ECDH Key Exchange

A detailed implementation of paper [Improving Bounds on Elliptic Curve Hidden Number Problem for ECDH Key Exchange](https://eprint.iacr.org/2022/1239.pdf). The paper provides a new algorithm to solve the Elliptic Curve Hidden Number Problem (ECHNP) for ECDH key exchange. The algorithm is based on the Copper-Smith method and the LLL algorithm. The paper provides a new bound for the ECHNP problem and shows that the new algorithm is more efficient than the previous algorithms.

Let's dive deep into the implementation of the algorithm!

## Reduce DH-Oracle to HNP

The ECHNP problem is a variant of the Hidden Number Problem (HNP) for elliptic curves. In a classic DH oracle, the attacker is given the public key $A = g^{a}, B = g^{b}$ and the goal is to recover the secret scalar $g^{ab}$ by querying the DH oracle : $\mathcal{O}_{A}(g^r) = \mathcal{MSB}_{k}(g^{ra})$ ($k$ msb or lsb leaks). This problem can be efficiently reduced to the HNP problem. 

**DH Reduction**
Given the public keys $A = g^{a}, B = g^{b}$ and DH-Oracle $\mathcal{O}_{A}(g^r) = \mathcal{MSB}(g^{ra})$, the attacker can compute the shared secret $x = g^{ab}$ as follows:

- Select a random scalar $r_i$ and compute $g^{r_i}$.
- Query the DH oracle $\mathcal{O}_{A}(g^r_i \cdot g^b) = \mathcal{MSB}_k(g^{r_i a} \cdot g^{ab})$ and denote the response as $y_i \le 2^k$.
- Compute the known values $t_i = g^{r_i a} = A^{r_i}$ and build equations $ \mathcal{MSB}_k(t_i \cdot x) = y_i$ which is exactly a HNP equation.
- Collect enough samples from oralce $\mathcal{O}$ until we can solve the HNP problem to recover the secret scalar $x = g^{ab}$.


Let us first define the ECDH leak oracle: the attacker is given the public key $A = [a]Q, B = [b]Q$ where $Q$ is the generator and the goal is to recover the secret scalar $[ab]P$ by querying the ECDH oracle : $\mathcal{O}_{A}(rQ) = \mathcal{MSB}([ra]Q)$ (or msb leak). The ECDH leakage model is more challenging because of the complexity of the elliptic curve group operations. The ECHNP problem is a variant of the HNP problem for elliptic curves and a reduction from ECDH oracle to ECHNP problem is not trivial. The paper provides a coppersmith method to solve the ECHNP problem efficiently.

## Reduce ECDH-Oracle to ECHNP

**Def 1. ECDH-MSB-Oracle**
Given the public keys $A = [a]Q, B = [b]Q$ in curve $E$ over $\mathbb{F}_p$ and generator $Q$, the ECDH-MSB-Oracle is $\mathcal{O}_{A}(rQ) = \mathcal{MSB}([ra]Q)$.

**Def 2. ECHNP**
Fix a prime $p$, a given curve $E$ over $\mathbb{F}_p$ , a given point $R \in E(\mathbb{F}_p)$ and a positive number $\delta$. Let $P \in E$ be a hidden point. Let $\mathcal{O}_{P, R}$ be an oracle that on input $m$ outputs the $\delta$ most significant bits of the $x$-coordinate of $P + [m]R$. That is, $\mathcal{O}_{P, R}(m) = \mathcal{MSB}_{\delta}(x_{P + [m]R})$. The goal is to recover the hidden point $P$, given query access to the oracle $\mathcal{O}_{P, R}$.

**ECDH Reduction**

Given the public keys $A = [a]Q, B = [b]Q$ in curve $E$ over $\mathbb{F}_p$, generator $Q$ and ECDH-MSB-Oracle $\mathcal{O}_{A}([r]Q) = \mathcal{MSB}([ra]Q)$, the attacker can compute the shared secret $[ab]Q$ as follows:

- Select a random scalar $r_i$ and compute $[r_i]Q$.
- Query the ECDH-MSB-Oracle $\mathcal{O}_{A}([r_i]Q + [b]Q) = \mathcal{MSB}(x_{[r_ia]Q + [ab]Q})$ and denote the response as $y_i \le 2^k$.
- Compute the known values $T = [r_i]A$ and build equations $ \mathcal{MSB}_k(x_{T + [ab]Q}) = y_i$ which is exactly $\mathcal{O}_{[ab]Q, A}(r_i) = \mathcal{MSB}_{\delta}(x_{P + [r_i]R})$.
- Collect enough samples from oralce $\mathcal{O}$ until we can solve the ECHNP problem to recover the secret point $[ab]Q$ (to be specific, the $x$-coordinate of $[ab]Q$).

Talk is cheap, let's implement the (EC)DH oracles!

In [1]:
# from ecdsa import NIST256p
# from ecdsa.ecdsa import Public_key
# from ecdsa.ellipticcurve import Point
from sage.all import EllipticCurve, GF, ZZ, Zmod, PolynomialRing, prod, matrix, QQ
import random
from Crypto.Util.number import getPrime

def secp256r1_curve():
    # https://neuromancer.sk/std/nist/P-256
    p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff
    K = GF(p)
    a = K(0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc)
    b = K(0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b)
    E = EllipticCurve(K, (a, b))
    G = E(0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296, 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5)
    E.set_order(0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551 * 0x1)
    return (E, G)
    
class DH():
    """
    A simple Diffie-Hellman and its oracle
    
    """
    def __init__(self, p, g):
        self.p = p
        self.nbit = ZZ(p).nbits()
        self.base_field = GF(p)
        self.g = self.base_field(g)
        self.a = random.randint(1, p-1)
        self.b = random.randint(1, p-1)
        self.A = self.g ** self.a
        self.B = self.g ** self.b
        self.shared_secret = self.g ** (self.a * self.b)

    def get_pubs(self):
        return (ZZ(self.A), ZZ(self.B))

    def get_shared_secret(self):
        return ZZ(self.shared_secret)
    
    def dh(self, C, alice=True):
        """Negotiate a shared secret between two parties

        Args:
            C (Integer): public key of the other party
            alice (bool, optional): If True, this party is Alice. Otherwise, Bob. Defaults to True. 

        Returns:
            Integer: shared secret
        """
        if alice:
            return ZZ(self.base_field(C) ** self.a)
        else:
            return ZZ(self.base_field(C) ** self.b)
    
    def oracle(self, C, kbit, msb=True, alice=True):
        """Oracle for the attacker to leak the dh output

        Args:
            C (Integer): public key of the other party
            kbit (Integer): number of bits to leak
            msb (bool, optional): leak from msb. Defaults to True. If False, leak from lsb.
            alice (bool, optional): If True, this party is Alice. Otherwise, Bob. Defaults to True. 
        """
        shared_secret = self.dh(C, alice)
        if msb:
            return ZZ(shared_secret >> (self.nbit - kbit))
        else:
            return ZZ(shared_secret % (2**kbit))
        
    def oracle_func(self, kbit, msb=True, alice=True):
        # return function for the oracle with fixed k, with input being only `C`
        return lambda C: self.oracle(C, kbit, msb, alice)
    
class ECDH:
    """
    A simple Elliptic Curve Diffie-Hellman and its oracle
    
    """
    
    def __init__(self, curve, G):
        self.curve = curve
        self.G = G
        self.nbit = ZZ(curve.base_field().order()).nbits()
        self.a = random.randint(1, curve.order()-1)
        self.b = random.randint(1, curve.order()-1)
        self.A = self.a * G
        self.B = self.b * G
        self.shared_secret = self.a * self.b * G
        
    def get_pubs(self):
        return (self.A, self.B)
    
    def get_shared_secret(self):
        return ZZ(self.shared_secret[0])
    
    def dh(self, C, alice=True):
        if alice:
            return ZZ((self.a * C)[0])
        else:
            return ZZ((self.b * C)[0])
        
    def oracle(self, C, kbit, msb=True, alice=True):
        shared_secret = self.dh(C, alice)
        if msb:
            return ZZ(shared_secret >> (self.nbit - kbit))
        else:
            return ZZ(shared_secret % (2**kbit))
        
    def oracle_func(self, kbit, msb=True, alice=True):
        # return function for the oracle with fixed k, with input being only `C`
        return lambda C: self.oracle(C, kbit, msb, alice)
    
    
def test_dh():
    print("[+] Testing DH-Oracle Implementation")
    p = getPrime(256)
    while p.bit_length() != 256:
        p = getPrime(256)
    g = 2
    dh = DH(p, g)
    A, B = dh.get_pubs()
    shared_secret = dh.get_shared_secret()
    assert shared_secret == dh.dh(B, True)
    assert shared_secret == dh.dh(A, False)
    print("[+] DH test passed")
    assert dh.oracle_func(128, True, True)(B) == shared_secret >> (dh.nbit - 128)
    assert dh.oracle_func(128, False, True)(B) == shared_secret % (2**128)
    assert dh.oracle_func(128, True, False)(A) == shared_secret >> (dh.nbit - 128)
    assert dh.oracle_func(128, False, False)(A) == shared_secret % (2**128)
    print("[+] DH-Oracle test passed")
    
def test_ecdh():
    print("[+] Testing ECDH-Oracle Implementation")
    E, G = secp256r1_curve()
    ecdh = ECDH(E, G)
    A, B = ecdh.get_pubs()
    shared_secret = ecdh.get_shared_secret()
    assert shared_secret == ecdh.dh(B, True)
    assert shared_secret == ecdh.dh(A, False)
    print("[+] ECDH test passed")
    assert ecdh.oracle_func(128, True, True)(B) == shared_secret >> (ecdh.nbit - 128)
    assert ecdh.oracle_func(128, False, True)(B) == shared_secret % (2**128)
    assert ecdh.oracle_func(128, True, False)(A) == shared_secret >> (ecdh.nbit - 128)
    assert ecdh.oracle_func(128, False, False)(A) == shared_secret % (2**128)
    print("[+] ECDH-Oracle test passed")
    
test_dh()
test_ecdh()

[+] Testing DH-Oracle Implementation
[+] DH test passed
[+] DH-Oracle test passed
[+] Testing ECDH-Oracle Implementation
[+] ECDH test passed
[+] ECDH-Oracle test passed


## ECHNP Details

Let's first focus on the elliptic curve and the additive group operations. Given a curve $E$ over $\mathbb{F}_p$ , a point $P = (x_P, y_P)$ and a point $Q = (x_Q, y_Q)$, let $P+Q = (x_R, y_R)$ be the sum of $P$ and $Q$. Then :

$$
\begin{aligned}
x_R &= \lambda^2 - x_P - x_Q \mod p \\
y_R &= \lambda(x_P - x_R) - y_P \mod p
\end{aligned}
$$

where $\lambda = \frac{y_Q - y_P}{x_Q - x_P} \mod p$ if $P \neq Q$ and $\lambda = \frac{3x_P^2 + a}{2y_P} \mod p$ if $P = Q$. The point doubling operation is $P + P = 2P$ and the point negation is $-P = (x_P, -y_P)$. The scalar multiplication is $[m]P = P + P + \cdots + P$ ($m$ times) using the double-and-add algorithm.

In the following sections, we will introduce how to use the coppersmith's method to solve the ECHNP problem efficiently.

### From ECHNP to modular polynomials

To apply the coppersmith's method, we need to convert the ECHNP problem to a small roots problem in modular polynomial. 

Let's consider the ECHNP oracle $\mathcal{O}_{P, R}(m) = \mathcal{MSB}_{\delta}(x_{P + [m]R})$. The goal is to recover the hidden point $P$ given query access to the oracle $\mathcal{O}_{P, R}$. The oracle $\mathcal{O}_{P, R}$ leaks the $\delta$ most significant bits of the $x$-coordinate of $P + [m]R$. 

**Eliminating $y_P$**. We consider the case $x_{P+Q}$ and $x_{P-Q}$ to obtain as simple polynomial as possible. Since $y^2 = x^3 + ax + b$ for $(x_P, y_P),(x_Q, y_Q)$, we can eliminate $y_P$ as follows:

$$
\begin{aligned}
x_{P+Q} + x_{P-Q} &= \lambda_{P+Q}^2 - x_P - x_Q + \lambda_{P-Q}^2 - x_P - x_Q \\
& = \frac{(y_Q - y_P)^2}{(x_Q - x_P)^2} + \frac{(y_Q + y_P)^2}{(x_Q - x_P)^2} - 2x_P - 2x_Q \\
& = 2 (\frac{(y_Q^2 + y_P^2)}{(x_Q - x_P)^2} - x_P - x_Q) \\
& = 2 (\frac{x_Q x_P^2 (a+x_Q^w)x_P + ax_Q + 2b}{(x_P-x_Q)^2}).
\end{aligned} \tag{1}
$$


We query the oracle $2n + 1$ with input $0, \pm m_1, \pm m_2, \cdots, m_n$ and collect the responses $h_i = \mathcal{O}_{{P}, R}(m_i) = {x_{P - [m_i]R}} - {e_i}, h(i)^\prime = \mathcal{O}_{{P}, R}(-m_i) = {x_{P - [m_i]R}} - {e_i^\prime}$. The responses $h_i$ and $h(i)^\prime$ are the $\delta$ most significant bits of the $x$-coordinate of ${P} + [m_i]R$ and ${P} - [m_i]R$ respectively. Thus, $|e_i| \le \frac{p}{2^{\delta+1}}$ and let $\tilde{h}_i=h_i+h_i^{\prime}$ and $\tilde{e}_i=e_i+e_i^{\prime}$, we have $\tilde{h}_i+\tilde{e}_i=x_{P+Q_i}+x_{P-Q_i}$, where $\left|\tilde{e}_i\right|<p / 2^\delta$ for $i=1, \cdots, n$. According to the equation $(1)$, we have: 

$$
\tilde{h}_i+\tilde{e}_i=2\left(\frac{x_{Q_i} x_P^2+\left(a+x_{Q_i}^2\right) x_P+a x_{Q_i}+2 b}{\left(x_P-x_{Q_i}\right)^2}\right), 1 \leq i \leq n .
$$

Moreover, we write $h_0=O_{P, R}(0)=\mathcal{MSB}_\delta\left(x_P\right)=x_P-e_0$, where $\left|e_0\right|<p / 2^{\delta+1}$. Hence, $\tilde{h}_i+\tilde{e}_i=2\left(\frac{x_{Q_i}\left(h_0+e_0\right)^2+\left(a+x_{Q_i}^2\right)\left(h_0+e_0\right)+a x_{Q_i}+2 b}{\left(h_0+e_0-x_{Q_i}\right)^2}\right)$. After multiplying by $\left(h_0+e_0-x_{Q_i}\right)^2$, we get $A_i+B_i e_0+C_i e_0^2+D_i \tilde{e}_i+E_i e_0 \tilde{e}_i+e_0^2 \tilde{e}_i=0 \bmod p$, $1 \leq i \leq n$, where known coefficients $A_i, B_i, C_i, D_i, E_i$ satisfy (in the field $\mathbb{F}_p$ )
$$
\begin{aligned}
& A_i=\left(\tilde{h}_i\left(h_0-x_{Q_i}\right)^2-2 h_0^2 x_{Q_i}-2\left(a+x_{Q_i}^2\right) h_0-2 a x_{Q_i}-4 b\right), \\
& B_i=2\left(\tilde{h}_i\left(h_0-x_{Q_i}\right)-2 h_0 x_{Q_i}-a-x_{Q_i}^2\right), C_i=\left(\tilde{h}_i-2 x_{Q_i}\right), \\
& D_i=\left(h_0-x_{Q_i}\right)^2, E_i=2\left(h_0-x_{Q_i}\right) .
\end{aligned}
$$

In short, we now have $n$ polynomials :

$$
\mathcal{F}_i\left(x_0, y_i\right):=A_i+B_i x_0+C_i x_0^2+D_i y_i+E_i x_0 y_i+x_0^2 y_i=0(\bmod p), 1 \leq i \leq n .
$$

where $(e_0,\tilde{e}_i)$ bounded by $(X, X), X =\frac{p}{2^\delta}$ is the desired small root of $\mathcal{F}_i$ i.e. $\mathcal{F}_i(e_0,\tilde{e}_i)=0$. 

Now the question is how to find the small roots of the polynomials $\mathcal{F}_i$ using the coppersmith's method.

In [2]:
def EcdhOracle2ECHNP(oracleA, B, Q):
    # oracleA: function to leak the shared secret oracle_a(C) = a*C
    # A, B: public keys of Alice and Bob
    # Q: generator point on the curve
    # returns the ECHNP oralce
    return lambda r : oracleA(B + r * Q)

def GenECHNP(oracleA, CurveParas, n, kbit, msb=True):
    # oracleA: function to leak the shared secret oracle_a(C) = a*C
    # CurveParas:
    # Curve: elliptic curve
    #   p : the modulus of the curve
    #   A, B: public keys of Alice and Bob
    #   Q: generator point on the curve
    # n: number of queries (total 2*n + 1)
    # kbit: number of bits to leak
    # returns the ECHNP samples
    (Curve, p, A, B, Q) = CurveParas
    oracle_ECHNP = EcdhOracle2ECHNP(oracleA, B, Q)
    pbit = ZZ(p).nbits()
    hs = []
    phs = []
    mhs = []
    xqs = []
    h0 = oracle_ECHNP(0) << (int(msb) * (pbit - kbit))
    for i in range(1, n + 1):
        h1 = ZZ(oracle_ECHNP(i) << (int(msb) * (pbit - kbit)))
        h2 = ZZ(oracle_ECHNP(-i) << (int(msb) * (pbit - kbit)))
        xq = ZZ((i * A)[0])
        xqs.append(xq)
        phs.append(h1)
        mhs.append(h2)
        hs.append(h1 + h2)
    return h0, hs, xqs, phs, mhs

def GetECHNPPolys(h0, hs, xqs, CurveParas):
    # hs: leaked bits
    # xqs: x-coordinates of the queries
    # CurveParas:
    # Curve: elliptic curve
    #   p : the modulus of the curve
    #   A, B: public keys of Alice and Bob
    #   Q: generator point on the curve
    # returns the ECHNP polynomials
    assert len(hs) == len(xqs), "Invalid input"
    n = len(hs)
    (Curve, p, A, B, Q) = CurveParas
    p = ZZ(p)
    a, b = ZZ(Curve.a4()), ZZ(Curve.a6())
    Aix = lambda hi, h0, xqi, p, a, b: ZZ((hi*(h0 - xqi)^2  - 2 * h0^2 * xqi - 2*(a + xqi^2)*h0 -\
                                        2*a*xqi - 4*b) % p)
    Bix = lambda hi, h0, xqi, p, a, b: ZZ(2 * (hi *(h0 - xqi) - 2* h0 * xqi - a - xqi^2) % p)
    Cix = lambda hi, h0, xqi, p, a, b: ZZ((hi - 2 * xqi) % p)
    Dix = lambda hi, h0, xqi, p, a, b: ZZ((h0 - xqi)^2 % p)
    Eix = lambda hi, h0, xqi, p, a, b: ZZ(2 * (h0 - xqi) % p)
    PR = PolynomialRing(GF(p), ["x", "y"])
    x, y = PR.gens()
    fxy = lambda hi, h0, xqi: Aix(hi, h0, xqi, p, a, b) * 1 +\
                              Bix(hi, h0, xqi, p, a, b) * x +\
                              Cix(hi, h0, xqi, p, a, b) * x^2 +\
                              Dix(hi, h0, xqi, p, a, b) * y + \
                              Eix(hi, h0, xqi, p, a, b) * x * y +\
                              x^2*y
    polys = []
    for i in range(n):
        hi = ZZ(hs[i])
        xqi = ZZ(xqs[i])
        f = -(2 * (xqi  *(h0 + x)^2 + (a + xqi^2)*(h0 + x) + a*xqi + 2*b) - (hi + y) * (h0 + x - xqi)^2)
        polys.append(PR(f))
        # polys.append(fxy(hi, h0, xqi))
        assert (fxy(hi, h0, xqi) == f)
    return polys

def test_GetECHNPPolys():
    E, G = secp256r1_curve()
    ecdh = ECDH(E, G)
    A, B = ecdh.get_pubs()
    p = ZZ(E.base_field().order())
    nbit = p.nbits()
    k = 140
    msb = True
    n = 10
    print("[+] Testing ECHNP Polynomials")
    print(f"[+] {msb = }")
    print(f"[+] {n = } {k = }")
    CurveParas = (E, E.base_field().order(), A, B, G)
    oracle = ecdh.oracle_func(k, msb, True)
    h0, hs, xqs, phs, mhs = GenECHNP(oracle, CurveParas, n, k, msb)
    polys = GetECHNPPolys(h0, hs, xqs, CurveParas)
    # full values
    oracle = ecdh.oracle_func(256, msb, True)
    H0, Hs, Xqs, pHs, mHs = GenECHNP(oracle, CurveParas, n, 256, msb)
    assert H0 >> (nbit - k) == h0 >> (nbit - k), "Invalid h0"
    assert all([h >> (nbit - k) == hh  >> (nbit - k) for h, hh in zip(phs, pHs)]), "Invalid phs"
    assert all([h >> (nbit - k) == hh  >> (nbit - k) for h, hh in zip(mhs, mHs)]), "Invalid mhs"
    assert all([x == xx for x, xx in zip(xqs, Xqs)]), "Invalid xqs"

    ebit = nbit - k
    e0 = H0 % 2**(ebit)
    es = [ZZ(h1 % 2**ebit) + ZZ(h2 % 2**ebit) for h1, h2 in zip(pHs, mHs)]
    # check the polynomial small roots
    for i in range(n):
        assert polys[i](e0, es[i]) % p == 0,  "Invalid polynomial"
    print("[+] All MSB ECHNP Polynomials Checked")
    
    msb = False
    print(f"[+] {msb = }")
    print(f"[+] {n = } {k = }")
    oracle = ecdh.oracle_func(k, msb, True)
    h0, hs, xqs, phs, mhs = GenECHNP(oracle, CurveParas, n, k, msb)
    polys = GetECHNPPolys(h0, hs, xqs, CurveParas)
    x, y = polys[0].parent().gens()
    lsb_polys = [poly(x*2^k, y*2^k) for poly in polys]
    # full values
    oracle = ecdh.oracle_func(256, msb, True)
    H0, Hs, Xqs, pHs, mHs = GenECHNP(oracle, CurveParas, n, 256, msb)

    e0 = H0 >> k
    es = [ZZ(h1 >> k) + ZZ(h2 >> k) for h1, h2 in zip(pHs, mHs)]
    # check the polynomial small roots
    for i in range(n):
        assert lsb_polys[i](e0, es[i]) % p == 0,  "Invalid polynomial"
    print("[+] All LSB ECHNP Polynomials Checked")
    print("[+] ECHNP Polynomials test passed")
    
def ECHNP_Polys(k = 150, n = 10):
    E, G = secp256r1_curve()
    ecdh = ECDH(E, G)
    A, B = ecdh.get_pubs()
    p = ZZ(E.base_field().order())
    nbit = p.nbits()
    ebit = nbit - k
    msb = True
    CurveParas = (E, E.base_field().order(), A, B, G)
    oracle = ecdh.oracle_func(k, msb, True)
    h0, hs, xqs, phs, mhs = GenECHNP(oracle, CurveParas, n, k, msb)
    polys = GetECHNPPolys(h0, hs, xqs, CurveParas)
    # full values
    oracle = ecdh.oracle_func(256, msb, True)
    H0, Hs, Xqs, pHs, mHs = GenECHNP(oracle, CurveParas, n, 256, msb)
    assert H0 >> (nbit - k) == h0 >> (nbit - k), "Invalid h0"
    assert all([h >> (nbit - k) == hh  >> (nbit - k) for h, hh in zip(phs, pHs)]), "Invalid phs"
    assert all([h >> (nbit - k) == hh  >> (nbit - k) for h, hh in zip(mhs, mHs)]), "Invalid mhs"
    assert all([x == xx for x, xx in zip(xqs, Xqs)]), "Invalid xqs"

    e0 = H0 % 2**(ebit)
    es = [ZZ(h1 % 2**ebit) + ZZ(h2 % 2**ebit) for h1, h2 in zip(pHs, mHs)]
    # check the polynomial small roots
    for i in range(n):
        assert polys[i](e0, es[i]) % p == 0,  "Invalid polynomial"
    return polys, e0, es

test_GetECHNPPolys()

[+] Testing ECHNP Polynomials
[+] msb = True
[+] n = 10 k = 140
[+] All MSB ECHNP Polynomials Checked
[+] msb = False
[+] n = 10 k = 140
[+] All LSB ECHNP Polynomials Checked
[+] ECHNP Polynomials test passed


## Shifting Polynomials

The core idea of the coppersmith's method is to find the small roots of the polynomials over $\mathbb{Z}_{p}$ by shifting the polynomials i.e. generating more useful polynomials (vectors) over an extended modulus : $\mathbb{Z}_{p^d}$ where $d$ is a custom parameter.

There are too many details to cover in this notebook, so I will provide a high-level overview of the coppersmith's method. 

In this paper, the authors provide a new lattice (improved shifting polynomials) for the ECHNP problem and show that the new shifting polynomials is more efficient than the previous ones. I will introduce both the original and the improved shifting polynomials.

### XHS20 Lattice

**Lattice $\mathcal{L}_{[\mathrm{XHS20}]}(n, d)$** 

For any fixed tuple $\left(i_0, i_1, \cdots, i_n\right) \in \mathcal{I}_{[\mathrm{XHS} 20]}(n, d)$, we construct polynomial $f_{i_0, i_1, \ldots, i_n}\left(x_0, y_1, \cdots, y_n\right)$ as follows.

- Case a: When $l=0$ and $0 \leq i_0 \leq 2 d$, define
    $$
    f_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right):=x_0^{i_0} .
    $$
- Case b: When $l=1$ and $0 \leq i_0 \leq 1$, define
    $$
    f_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right):=x_0^{i_0} y_1^{i_1} \cdots y_n^{i_n} .
    $$
- Case $\mathbf{c}$ : When $1 \leq l \leq d$ and $2 l \leq i_0 \leq 2 d$, define
    $$
    f_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right):=x_0^{i_0-2 l} \mathcal{F}_1^{i_1} \cdots \mathcal{F}_n^{i_n} .
    $$
- Case d: When $2 \leq l \leq d$ and $0 \leq i_0 \leq 2 l-1$, define
    $$
    f_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right):=\sum_{u=1}^l \sum_{v=0}^1 w_{i_0+1, u+l v} \cdot x_0^v \mathcal{F}_{j_1} \cdots \mathcal{F}_{j_{u-1}} y_{j_u} \mathcal{F}_{j_{u+1}} \cdots \mathcal{F}_{j_l},
    $$
    where $\mathcal{F}_i\left(x_0, y_i\right)=A_i+B_i x_0+C_i x_0^2+D_i y_i+E_i x_0 y_i+x_0^2 y_i=0(\bmod p)$ for $1 \leq i \leq n$ defined in (6), integers $j_1, \cdots, j_l$ are defined in Lemma 3 , and $w_{i_0+1, u+l v}$ is element of the $\left(i_0+1\right)$-th row and the $(u+l v)$-th column of the matrix $\mathbf{W}_{j_1, \cdots, j_l}$, which is also defined in Lemma 3.


**Lemma 3**. Let $i_1, \cdots, i_n$ be integers satisfying $0 \leq i_1, \cdots, i_n \leq 1$. Denote $l=i_1+\cdots+i_n$, where $2 \leq l \leq n$. Let $j_1, \cdots, j_l$ be integers satisfying $1 \leq j_1<$ $\cdots<j_l \leq n$ and $y_{j_1} \cdots y_{j_l}=y_1^{\overline{i_1}} \cdots y_n^{i_n}$. Let a $2 l \times 2 l$ integer matrix $\mathbf{M}_{j_1, \cdots, j_l}$ be the following coefficient matrix:
$$
\left(\begin{array}{c}
\prod_{u \neq 1}\left(x_0^2+E_{j_u} x_0+D_{j_u}\right) \\
\vdots \\
\prod_{u \neq l}\left(x_0^2+E_{j_u} x_0+D_{j_u}\right) \\
x_0 \prod_{u \neq 1}\left(x_0^2+E_{j_u} x_0+D_{j_u}\right) \\
\ddots \\
x_0 \prod_{u \neq l}\left(x_0^2+E_{j_u} x_0+D_{j_u}\right)
\end{array}\right)=\mathbf{M}_{j_1, \cdots, j_l}\left(\begin{array}{c}
1 \\
\vdots \\
x_0^{l-1} \\
x_0^l \\
\vdots \\
x_0^{2 l-1}
\end{array}\right) \bmod p^{l-1},
$$
where integers $D_{j_u}$ and $E_{j_u}$ are the coefficients in the polynomial $\mathcal{F}_{j_u}=A_{j_u}+$ $B_{j_u} x_0+C_{j_u} x_0^2+D_{j_u} y_{j_u}+E_{j_u} x_0 y_{j_u}+x_0^2 y_{j_u}$ for $1 \leq u \leq l$. Then the matrix $\mathbf{M}_{j_1, \cdots, j_l}$ is invertible over $\mathbb{Z}_{p^{l-1}}$. Denote $\mathbf{W}_{j_1, \cdots, j_l}$ as its inverse matrix. Hence,
$$
\mathbf{W}_{j_1, \cdots, j_l} \cdot \mathbf{M}_{j_1, \cdots, j_l}=I_{2 l} \bmod p^{l-1}
$$
where $I_{2 l}$ is the $2 l \times 2 l$ identity matrix.

Lemma $4$. Based on the order defined in original paper, the monomial $x_0^{i_0} y_1^{i_1} \cdots y_n^{i_n}$ is the leading term of the polynomial $f_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right)$ for $\left(i_0, i_1, \cdots, i_n\right) \in$ $\mathcal{I}_{[\mathrm{XHS} 20]}(n, d)$. Let
$$
F_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right):=\left\{\begin{array}{l}
p^{d+1-l} f_{i_0, i_1, \cdots, i_n} \text { for } 1 \leq l \leq d, 0 \leq i_0 \leq 2 l-1 \\
p^{d-l} f_{i_0, i_1, \cdots, i_n} \text { for } 0 \leq l \leq d, 2 l \leq i_0 \leq 2 d
\end{array}\right.
$$

One can verify that for all $F_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right)$, they have small roots $(e_0, \tilde{e}_1, \cdots, \tilde{e}_{n})$ over $\mathbb{Z}_{p^d}$ i.e. $F_{i_0, i_1, \cdots, i_n}\left(e_0, \tilde{e}_1, \cdots, \tilde{e}_{n}\right)=0 \bmod p^d$.

Let $\mathcal{L}_{[\mathrm{XHS20]}}(n, d)$ be the lattice which is spanned by the coefficient vectors of polynomials

There are details in the paper that I have skipped for brevity. The interested reader can refer to the paper for more details about why these complex polynomials are constructed, the determinant of the final matrix constructed from $\mathcal{L}_{[\mathrm{XHS20}]}(n, d)$, the bound for the small roots, etc (details about lattice cryptanalysis).


### The Improved XHS22 Lattice

**XHS22 Lattice $\mathcal{L}(n, d, t)$**

Let $\mathcal{I}(n, d, t)$ be an index set which is equal to $\mathcal{I}(n, d, t)=\mathcal{I}_1 \cup \mathcal{I}_2$, where

$$
\begin{aligned}
& \mathcal{I}_1:=\left\{\left(i_0, i_1, \cdots, i_n\right) \mid 0 \leq i_0 \leq 2 d-1,0 \leq i_1, \cdots, i_n \leq 1,0 \leq l \leq d\right\}, \\
& \mathcal{I}_2:=\left\{\left(i_0, i_1, \cdots, i_n\right) \mid 0 \leq i_0 \leq t, 0 \leq i_1, \cdots, i_n \leq 1, l=d+1\right\} .
\end{aligned}
$$

Here, $1 \leq d<n, 0 \leq t \leq 2 d-1$ and $l=i_1+\cdots+i_n$ satisfying $0 \leq l \leq d+1$.
Remark 1. According to (12), we get that the index set $\mathcal{I}_{[\mathrm{XHS20]}}(n, d)$ equals
$$
\left\{\left(i_0, i_1, \cdots, i_n\right) \mid 0 \leq i_0 \leq 2 d, 0 \leq i_1, \cdots, i_n \leq 1,0 \leq l \leq d\right\} .
$$
It is obvious that $\mathcal{I}_1$ is a subset of $\mathcal{I}_{[\mathrm{XHS} 20]}(n, d)$, whereas $\mathcal{I}_2$ is not.



Based on $F_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right)$ in Lemma 4, we construct the polynomial $G_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right)$ as follows.

- Case A: For any given $\left(i_0, i_1, \cdots, i_n\right) \in \mathcal{I}_1$, we define
    $$
    G_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right)=F_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right) .
    $$
    Since $F_{i_0, i_1, \cdots, i_n}\left(e_0, \widetilde{e}_1, \cdots, \widetilde{e}_n\right)=0 \bmod p^d$, we have $G_{i_0, i_1, \cdots, i_n}\left(e_0, \widetilde{e}_1, \cdots, \widetilde{e}_n\right)=$ $0 \bmod p^d$.
- Case B: For any given $\left(i_0, i_1, \cdots, i_n\right) \in \mathcal{I}_2$, we define
    $$
    G_{i_0, i_1, \cdots, i_n}\left(x_0, y_1, \cdots, y_n\right)=\left(H_{i_0, i_1, \cdots, i_n}+J_{i_0, i_1, \cdots, i_n}+K_{i_0, i_1, \cdots, i_n}\right) \bmod p^d,
    $$
    which is considered to be the corresponding polynomial over $\mathbb{Z}$. Without loss of generality, we let $j_1, \cdots, j_{d+1}$ be integers satisfying $1 \leq j_1 \leq \cdots \leq j_{d+1} \leq n$ and $y_{j_1} y_{j_2} \cdots y_{j_{d+1}}=y_1^{i_1} y_2^{i_2} \cdots y_n^{i_n}$, and
    $$
    \begin{aligned}
    & H_{i_0, i_1, \cdots, i_n}=\sum_{u=1}^{d+1} \sum_{v=0}^1 w_{i_0+1, u+v(d+1)} \cdot x_0^v \mathcal{F}_{j_1} \cdots \mathcal{F}_{j_{u-1}} y_{j_u} \mathcal{F}_{j_{u+1}} \cdots \mathcal{F}_{j_{d+1}}, \\
    & J_{i_0, i_1, \cdots, i_n}=\sum_{u=1}^{d+1} \sum_{v=0}^1 w_{i_0+1, u+v(d+1)} \cdot x_0^v \mathcal{F}_{j_1} \cdots \mathcal{F}_{j_{u-1}} C_{j_u} \mathcal{F}_{j_{u+1}} \cdots \mathcal{F}_{j_{d+1}}, \\
    & K_{i_0, i_1, \cdots, i_n}=\sum_{u=1}^{d+1} w_{i_0+1, u+(d+1)} \cdot \mathcal{F}_{j_1} \cdots \mathcal{F}_{j_{u-1}}\left(B_{j_u}-C_{j_u} E_{j_u}\right) \mathcal{F}_{j_{u+1}} \cdots \mathcal{F}_{j_{d+1}},
    \end{aligned}
    $$
    where the integers $B_{j_u}, C_{j_u}$ and $E_{j_u}$ are the coefficients in the polynomial $\mathcal{F}_{j_u}=A_{j_u}+B_{j_u} x_0+C_{j_u} x_0^2+D_{j_u} y_{j_u}+E_{j_u} x_0 y_{j_u}+x_0^2 y_{j_u}$ for $1 \leq u \leq d+1$, and the integer $w_{i_0+1, m}(1 \leq m \leq 2 d+2)$ is the $m$-th component of the $\left(i_0+1\right)$-th row vector in the inverse matrix $\mathbf{W}_{j_1, \cdots, j_{d+1}}$, which is defined in Lemma 3 .

For Case B, the desired vector $\left(e_0, \widetilde{e}_1, \cdots, \widetilde{e}_n\right)$ is common root of $H_{i_0, i_1, \cdots, i_n}$, $J_{i_0, i_1, \cdots, i_n}$ and $K_{i_0, i_1, \cdots, i_n}$ modulo $p^d$. Hence, $G_{i_0, i_1, \cdots, i_n}\left(e_0, \widetilde{e}_1, \cdots, \widetilde{e}_n\right) \stackrel{=}{=}$ $0 \bmod p^d$.

### Implementation

The following codes shows how to get all the shifting polynomial vectors for the ECHNP problem using the both XHS20 lattice and improved XHS22 lattice.

In [3]:
def XHS20_Lattice_l_i(d, l, i0, js, x0, ys, polys, W=None):
    # assert 0 <= l <= d, "Invalid l"
    # assert 0 <= i0 <= 2*d, "Invalid i0"
    # assert len(js) == l, "Invalid positions"
    # case a
    if l == 0:
        return x0^i0
    # case b, l = 1
    elif l == 1 and i0 <= 1:
        return x0^i0 * ys[js[0]]
    # case c
    elif l >= 1 and i0 >= 2*l:
        F = prod(polys[j] for j in js)
        return x0^(i0 - 2*l) * F
    # case d
    elif l >= 2 and i0 <= 2*l -1:
        res =  0
        F = prod(polys[j] for j in js)
        for u in range(l):
            # F = prod(polys[j] for j in js if j != js[u])
            P = F // polys[js[u]]
            for v in range(2):
                res += W[i0, u + l * v] * (x0^v) * (P) * ys[js[u]]
        return res
    else:
        assert False,  f"Unknown Case {d = } { l = } {i0 = }"
        
def XHS22_Lattice_l_i_t(d, l, i0, t, js, x0, ys, polys, W=None, C=None, B=None, E=None):
    if i0 <= 2*d - 1 and l <= d:
        # case a : the same as XHS20 Lattice
        return XHS20_Lattice_l_i(d, l, i0, js, x0, ys, polys, W)
    elif i0 <= t and l == d + 1:
        res =  0
        F = prod(polys[j] for j in js)
        for u in range(l):
            # F = prod(polys[j] for j in js if j != js[u])
            P = F // polys[js[u]]
            for v in range(2):
                # H poly items
                res += W[i0, u + l * v] * (x0^v) * (P) * ys[js[u]]
                # J poly items
                res += W[i0, u + l * v] * (x0^v) * (P) * C[js[u]]
            # K poly items
            res += W[i0, u + l] * (P) * (B[js[u]] - C[js[u]] * E[js[u]])
        return res
    else:
        print(f"[+] unhanled case {d = } {l = } {i0 = } {t = }")
        return None

    
def M_polys(xpolys, positions):
    S = prod(xpolys[pos] for pos in positions)
    x = xpolys[0].parent().gens()[0]
    polylist1 = []
    polylist2 = []
    for i in range(len(positions)):
        tmp = S // xpolys[positions[i]]
        assert tmp * xpolys[positions[i]] == S
        polylist1.append(tmp)
        polylist2.append(x * tmp)
    return polylist1 + polylist2
    
def fast_coef_mat(polys):
    # must define the order of the monomials
    # mat = matrix(GF(2), len(polys), len(monos))
    monos = set()
    for poly in polys:
        monos.update(poly.monomials())
    monos = sorted(list(monos))
    # print(monos)
    mono_to_index = {}
    for i, mono in enumerate(monos):
        mono_to_index[mono] = i
    mat = [[0] * len(monos) for i in range(len(polys))]
    
    for i, f in (list(enumerate(polys))):
        for coeff, mono in f:
            # mat[i,mono_to_index[mono]] = 1
            mat[i][mono_to_index[mono]] = coeff
    return mat, monos

def _coeff_mat_uni(mono_to_index, monos, polys):
    # mat = matrix(GF(2), len(polys), len(monos))
    mat = [[0] * len(monos) for i in range(len(polys))]
    pr = polys[0].parent()
    x = pr.gens()[0]
    for i, f in (list(enumerate(polys))):
        for coeff in f:
            # mat[i,mono_to_index[mono]] = 1
            mat[i][mono_to_index[pr(x^i)]] = coeff
    return mat

def coeff_mat_uni(polys, maxdeg=None):
    if maxdeg is None:
        maxdeg = max([poly.degree() for poly in polys])
    mat = [[0] * (maxdeg + 1) for i in range(len(polys))]
    pr = polys[0].parent()
    x = pr.gens()[0]
    seq = Sequence(polys, pr)
    for i, f in (list(enumerate(polys))):
        for j,coeff in enumerate(f):
            mat[i][j] = coeff
    return mat
    

And then build the lattice from the polynomials' coefficients and do the LLL reduction to find the reduced basis. We can reconstruct reduced polynomials from the reduced basis which may have the desired small roots in integer ring!

In [4]:
from findRootsZZ import find_roots_gcd, find_roots_groebner, find_roots_resultants, find_roots_variety
from rootfind_ZZ import rootfind_ZZ, JACOBIAN, HENSEL, TRIANGULATE, GROEBNER 

def XHS20_Lattice_Polys(n, d, p, polys, remove_d=False):
    # remove_d : for XHS22 lattice
    assert d <= n, "Invalid d"
    from itertools import combinations
    poly_ring = PolynomialRing(Zmod(p^d), [f"x0"] + [f"y{i}" for i in range(1, n+1)], order='negdegrevlex')
    poly_ring_uni = PolynomialRing(Zmod(p^d),"x")
    # polys = [ poly.change_ring(Zmod(p^d))  for poly in poly]
    xs = poly_ring.gens()
    x0, ys = xs[0], xs[1:]
    x = poly_ring_uni.gens()[0]
    # change polys's variables to x, y1, y2, ..., y_n
    spolys = []
    for poly, y in zip(polys, ys):
        spolys.append(poly_ring(poly(x0, y)))
    xpolys = []
    # save only y_i(x^2 + Ex + D) from poly = A + Bx + Cx^2 + Dy + Exy + x^2y
    for poly in polys:
        xx, yy = poly.parent().gens()
        D = ZZ(poly.coefficient({xx: 0, yy: 1}))
        E = ZZ(poly.coefficient({xx: 1, yy: 1}))
        xpolys.append(poly_ring_uni(D + E*x + x^2))
    monos = [poly_ring_uni(x^i) for i in range(2*d)]    
    res  = []
    Ws = {}
    i0_bound = 2 * d if remove_d else 2*d + 1
    for i0 in range(0, i0_bound):
        for l in range(0, d + 1):
            for i_pos in combinations(list(range(n)), l):
                if 2 <= l and i0 <= 2*l -1:
                    if str(i_pos) not in Ws:
                        mpolys = M_polys(xpolys, i_pos)
                        M = matrix(Zmod(p^d), coeff_mat_uni(mpolys))
                        # M = fast_coef_mat_uni(mono_to_index, monos, mpolys)
                        xmonos = vector(monos[:2*l])
                        xxpolys = M * xmonos
                        for p1, p2 in zip(mpolys, xxpolys):
                            assert p1 == p2
                        Ws[str(i_pos)] = M^(-1)
                    tmp_poly = XHS20_Lattice_l_i(d, l, i0, i_pos, x0, ys, spolys, Ws[str(i_pos)])
                else:
                    tmp_poly = XHS20_Lattice_l_i(d, l, i0, i_pos, x0, ys, spolys)
                if 1 <= l <= d and  0 <= i0 <= 2*l - 1:
                    res.append(p^(d + 1 - l) * tmp_poly.change_ring(ZZ))
                elif i0 >= 2*l:
                    res.append(p^(d - l) * tmp_poly.change_ring(ZZ))
    return res

def _XHS22_Lattice_Polys(n, d, t, p, polys):
    assert d <= n, "Invalid d"
    assert t <= 2*d - 1
    from itertools import combinations
    poly_ring = PolynomialRing(Zmod(p^d), [f"x0"] + [f"y{i}" for i in range(1, n+1)], order='negdegrevlex')
    poly_ring_uni = PolynomialRing(Zmod(p^d),"x")
    xs = poly_ring.gens()
    x0, ys = xs[0], xs[1:]
    x = poly_ring_uni.gens()[0]
    
    # change polys's variables to x, y1, y2, ..., y_n
    spolys = []
    for poly, y in zip(polys, ys):
        spolys.append(poly_ring(poly(x0, y)))
        
    # save only y_i(x^2 + Ex + D) from poly = A + Bx + Cx^2 + Dy + Exy + x^2y
    xpolys = []
    Bs = []
    Cs = []
    Es = []
    for poly in polys:
        xx, yy = poly.parent().gens()
        B = ZZ(poly.coefficient({xx: 1, yy: 0}))
        C = ZZ(poly.coefficient({xx: 2, yy: 0}))
        D = ZZ(poly.coefficient({xx: 0, yy: 1}))
        E = ZZ(poly.coefficient({xx: 1, yy: 1}))
        Bs.append(B)
        Cs.append(C)
        Es.append(E)
        xpolys.append(poly_ring_uni(D + E*x + x^2))
    # monos = [poly_ring_uni(x^i) for i in range(2*d)]    
    res  = []
    Ws = {}
    for i0 in range(0, 2*d):
        for l in range(0, d + 2):
            for i_pos in combinations(list(range(n)), l):
                if (2 <= l <= d and i0 <= 2*l -1):
                    if str(i_pos) not in Ws:
                        mpolys = M_polys(xpolys, i_pos)
                        M = matrix(Zmod(p^d), coeff_mat_uni(mpolys))
                        Ws[str(i_pos)] = M^(-1)
                    tmp_poly = XHS22_Lattice_l_i_t(d, l, i0, t, i_pos, x0, ys, spolys, Ws[str(i_pos)])
                elif i0 <= t and l == d + 1:
                    if str(i_pos) not in Ws:
                        mpolys = M_polys(xpolys, i_pos)
                        M = matrix(Zmod(p^d), coeff_mat_uni(mpolys))
                        Ws[str(i_pos)] = M^(-1)
                    tmp_poly = XHS22_Lattice_l_i_t(d, l, i0, t, i_pos, x0, ys, spolys, Ws[str(i_pos)], Cs, Bs, Es)
                else:
                    tmp_poly = XHS22_Lattice_l_i_t(d, l, i0, t, i_pos, x0, ys, spolys)
                if tmp_poly == None:
                    break
                if l == d + 1:
                    res.append(tmp_poly.change_ring(ZZ))
                if 1 <= l <= d and  0 <= i0 <= 2*l - 1:
                    res.append(p^(d + 1 - l) * tmp_poly.change_ring(ZZ))
                elif i0 >= 2*l:
                    res.append(p^(d - l) * tmp_poly.change_ring(ZZ))
    return res    

def XHS22_Lattice_Polys(n, d, t, p, polys):
    assert d <= n, "Invalid d"
    assert t <= 2*d - 1
    from itertools import combinations
    poly_ring = PolynomialRing(Zmod(p^d), [f"x0"] + [f"y{i}" for i in range(1, n+1)], order='negdegrevlex')
    poly_ring_uni = PolynomialRing(Zmod(p^d),"x")
    xs = poly_ring.gens()
    x0, ys = xs[0], xs[1:]
    x = poly_ring_uni.gens()[0]
    
    # change polys's variables to x, y1, y2, ..., y_n
    spolys = []
    for poly, y in zip(polys, ys):
        spolys.append(poly_ring(poly(x0, y)))
        
    # save only y_i(x^2 + Ex + D) from poly = A + Bx + Cx^2 + Dy + Exy + x^2y
    xpolys = []
    Bs = []
    Cs = []
    Es = []
    for poly in polys:
        xx, yy = poly.parent().gens()
        B = ZZ(poly.coefficient({xx: 1, yy: 0}))
        C = ZZ(poly.coefficient({xx: 2, yy: 0}))
        D = ZZ(poly.coefficient({xx: 0, yy: 1}))
        E = ZZ(poly.coefficient({xx: 1, yy: 1}))
        Bs.append(B)
        Cs.append(C)
        Es.append(E)
        xpolys.append(poly_ring_uni(D + E*x + x^2))
    # monos = [poly_ring_uni(x^i) for i in range(2*d)]    
    res  = []
    Ws = {}
    res += XHS20_Lattice_Polys(n, d, p, polys, remove_d=True)
    for i0 in range(0, t + 1):
        l = d + 1
        for i_pos in combinations(list(range(n)), l):
            if str(i_pos) not in Ws:
                mpolys = M_polys(xpolys, i_pos)
                M = matrix(Zmod(p^d), coeff_mat_uni(mpolys))
                Ws[str(i_pos)] = M^(-1)
            tmp_poly = XHS22_Lattice_l_i_t(d, l, i0, t, i_pos, x0, ys, spolys, Ws[str(i_pos)], Cs, Bs, Es)
            if tmp_poly == None:
                break
            res.append(tmp_poly.change_ring(ZZ))
    return res

def flatter(M):
    from subprocess import check_output
    from re import findall
    # compile https://github.com/keeganryan/flatter and put it in $PATH
    z = "[[" + "]\n[".join(" ".join(map(str, row)) for row in M) + "]]"
    ret = check_output(["flatter"], input=z.encode())
    return matrix(M.nrows(), M.ncols(), map(int, findall(b"-?\\d+", ret)))

def matrix_overview(mat):
    # 0 -> space, non-zero -> x
    for row in mat:
        for num in row:
            if num == 0:
                print("0", end="")
            else:
                print("x", end="")
        print()

def coppersmith_multivarivate_reduced_poly(polys, bounds):
    qq_poly_ring = polys[0].parent().change_ring(QQ)
    polys = [poly.change_ring(QQ) for poly in polys]
    mat, monomials = fast_coef_mat(polys)
    B = matrix(ZZ, mat)
    # matrix_overview(B)
    # print(f"[+] {B.is_triangular('lower') = }")
    # print(f"[+] {B.is_triangular('upper') = }")
    print(f"[+] {B.dimensions() = }. LLLing...")
    factors = [monomial(*bounds) for monomial in monomials]
    for i, factor in enumerate(factors):
        B.rescale_col(i, factor)
    try:
        # doing flatter, much faster than LLL
        B = flatter(B.dense_matrix())
        print("[+] Flatter-LLL Done")
    except:
        # native LLL reduction
        B = B.LLL()
        print("[+] Native-LLL Done")
    B = B.change_ring(QQ)
    for i, factor in enumerate(factors):
        B.rescale_col(i, 1/factor)
    monomials = vector(monomials)
    return list(filter(None, B*monomials))

def ECDH_To_ECHNP_Polys(n, k):
    E, G = secp256r1_curve()
    ecdh = ECDH(E, G)
    A, B = ecdh.get_pubs()
    p = ZZ(E.base_field().order())
    nbit = p.nbits()
    ebit = nbit - k
    msb = True
    CurveParas = (E, E.base_field().order(), A, B, G)
    oracle = ecdh.oracle_func(k, msb, True)
    h0, hs, xqs, phs, mhs = GenECHNP(oracle, CurveParas, n, k, msb)
    polys = GetECHNPPolys(h0, hs, xqs, CurveParas)
    # full values
    oracle = ecdh.oracle_func(256, msb, True)
    H0, Hs, Xqs, pHs, mHs = GenECHNP(oracle, CurveParas, n, 256, msb)
    assert H0 >> (nbit - k) == h0 >> (nbit - k), "Invalid h0"
    assert all([h >> (nbit - k) == hh  >> (nbit - k) for h, hh in zip(phs, pHs)]), "Invalid phs"
    assert all([h >> (nbit - k) == hh  >> (nbit - k) for h, hh in zip(mhs, mHs)]), "Invalid mhs"
    assert all([x == xx for x, xx in zip(xqs, Xqs)]), "Invalid xqs"

    e0 = H0 % 2**(ebit)
    es = [ZZ(h1 % 2**ebit) + ZZ(h2 % 2**ebit) for h1, h2 in zip(pHs, mHs)]
    # check the polynomial small roots
    for i in range(n):
        assert polys[i](e0, es[i]) % p == 0,  "Invalid polynomial"
    return polys, e0, es

PARI stack size set to 1073741824 bytes, maximum size set to 1073741824


## The Coppersmith Process with Details

Some internel-process information about the coppersmith method.

## XHS20 Lattice

In [5]:
def test_Coppersmallroot_With_Details_Secp256_XHS20(kbit = 185, n = 5, d = 2):
    E, G = secp256r1_curve()
    p = ZZ(E.base_field().order())
    pbit = p.nbits()
    ebit = pbit - kbit
    msb = True
    print(f"[+] The basic parameters {pbit = } {kbit = } {msb = }")
    print(f"[+] Try to use lattice parameters {n = } {d = } with XHS20 Lattice")
    polys, e, es = ECHNP_Polys(kbit, n)
    roots = [e] + es
    bounds = [2**ebit] + [2**(ebit+1)] * n
    count_poly = lambda d, n : (2*d + 1) * sum(binomial(n, l) for l in range(0,d+1))
    copper_polys = XHS20_Lattice_Polys(n, d, p, polys)
    total_poly_num = count_poly(d, n)
    print(f"[+] Shifted Polynomials test passed {len(copper_polys) = } {total_poly_num == len(copper_polys)}")
    res = []    
    tmp = []
    useful_copper_polys = []
    for poly in copper_polys:
        if poly == 0:
            continue
        useful_copper_polys.append(poly)
        res.append(poly(*roots) % p**d == 0)       
        tmp.append(poly(*roots) % p**(d+1) == 0)
        
    print(f"[+] Check Results (should be all true) {res.count(True) = } { res.count(False) = }")
    print(f"[+] Check tmp (should be all false) {tmp.count(True) = } { tmp.count(False) = }")
    polys = coppersmith_multivarivate_reduced_poly(useful_copper_polys, bounds)
    # check these polys
    good_polys_idx = []
    for i, poly in enumerate(polys):
        poly = poly.change_ring(ZZ)
        assert poly(*roots) % p**d == 0, "not copper poly"
        # check wether the poly has the original roots in ZZ
        if poly(*roots) == 0:
            good_polys_idx.append(i)
    print(f"[+] find {len(good_polys_idx)} polynomials with the desired roots in ZZ")
    print(f"[+] the good polynomials indexes : {good_polys_idx}")
    
    print("[+] try the groebner method to find roots")
    roots = find_roots_groebner(polys[0].parent(), polys)
    for root in roots:        
        print(f"[+] find : {root = }")
        
    print("[+] try the variety method to find roots")
    roots = find_roots_variety(polys[0].parent(), polys)
    for root in roots:
        print(f"[+] find : {root = }")
        
    print("[+] try the gcd method to find roots")
    roots = find_roots_gcd(polys[0].parent(), polys)
    for root in roots:
        print(f"[+] find : {root = }")
    print()
    # print("[+] try the resultant method to find roots")
    # roots = find_roots_resultants(polys[0].parent().gens(), polys)
    # for root in roots:
    #    print(f"[+] find : {root = }")

In [6]:
test_Coppersmallroot_With_Details_Secp256_XHS20()

[+] The basic parameters pbit = 256 kbit = 185 msb = True
[+] Try to use lattice parameters n = 5 d = 2 with XHS20 Lattice
[+] Shifted Polynomials test passed len(copper_polys) = 80 True
[+] Check Results (should be all true) res.count(True) = 80  res.count(False) = 0
[+] Check tmp (should be all false) tmp.count(True) = 0  tmp.count(False) = 80
[+] B.dimensions() = (80, 80). LLLing...
[+] Flatter-LLL Done
[+] find 37 polynomials with the desired roots in ZZ
[+] the good polynomials indexes : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
[+] try the groebner method to find roots
[+] find : root = {x0: 996512582924720619136, y1: 2381160775022383504659, y2: 2258839143789364811654, y3: 605010703946831370297, y4: 3753875062785592886015, y5: 4025030256658198411213}
[+] try the variety method to find roots
[+] try the gcd method to find roots



## XHS22 Lattice

In [7]:
def test_Coppersmallroot_With_Details_Secp256_XHS22(kbit = 172, n = 5, d = 2, t = 1, first_n = 20):
    E, G = secp256r1_curve()
    p = ZZ(E.base_field().order())
    pbit = p.nbits()
    ebit = pbit - kbit

    msb = True
    print(f"[+] The basic parameters {pbit = } {kbit = } {msb = }")
    print(f"[+] Try to use lattice parameters {n = } {d = } {t = } with XHS22 Lattice")
    polys, e, es = ECHNP_Polys(kbit, n)
    roots = [e] + es
    bounds = [2**ebit] + [2**(ebit+1)] * n
    count_poly = lambda n, d, t : (2*d) * sum(binomial(n, l) for l in range(0,d+1)) + (t + 1) * binomial(n, d + 1)
    copper_polys = XHS22_Lattice_Polys(n, d, t, p, polys)
    total_poly_num = count_poly(n, d, t)
    print(f"[+] Shifted Polynomials test passed {len(copper_polys) = } {total_poly_num = }")
    res = []    
    tmp = []
    useful_copper_polys = []
    for poly in copper_polys:
        if poly == 0:
            continue
        useful_copper_polys.append(poly)
        res.append(poly(*roots) % p**d == 0)       
        tmp.append(poly(*roots) % p**(d+1) == 0)
        
    print(f"[+] Check Results (should be all true) {res.count(True) = } { res.count(False) = }")
    print(f"[+] Check tmp (should be all false) {tmp.count(True) = } { tmp.count(False) = }")
    polys = coppersmith_multivarivate_reduced_poly(useful_copper_polys, bounds)
    # check these polys
    good_polys_idx = []
    for i, poly in enumerate(polys):
        poly = poly.change_ring(ZZ)
        assert poly(*roots) % p**d == 0, "not copper poly"
        # check wether the poly has the original roots in ZZ
        if poly(*roots) == 0 and poly % p**d != 0:
            good_polys_idx.append(i)
    print(f"[+] find {len(good_polys_idx)} polynomials with the desired roots in ZZ")
    print(f"[+] the good polynomials indexes : {good_polys_idx}")
    if first_n is None:
        first_n = len(polys)
    final_polys = polys[:first_n]
        
    print(f"[+] select first {first_n} of {len(polys)} polynomials to find the roots in ZZ")
    # print("[+] try the groebner method to find roots")
    # roots = find_roots_groebner(polys[0].parent(), final_polys)
    # for root in roots:        
    #     print(f"[+] find : {root = }")
        
    print("[+] try the multiple method to find roots")
    roots = rootfind_ZZ(final_polys, bounds)
    print(f"[+] find : {roots = }")
    
    print("[+] try the variety method to find roots")
    roots = find_roots_variety(polys[0].parent(), final_polys)
    for root in roots:
        print(f"[+] find : {root = }")
        
    print("[+] try the gcd method to find roots")
    roots = find_roots_gcd(polys[0].parent(), final_polys)
    for root in roots:
        print(f"[+] find : {root = }")
    print()

In [8]:
test_Coppersmallroot_With_Details_Secp256_XHS22(kbit = 165, n = 5, d = 3, t = 2)

[+] The basic parameters pbit = 256 kbit = 165 msb = True
[+] Try to use lattice parameters n = 5 d = 3 t = 2 with XHS22 Lattice
[+] Shifted Polynomials test passed len(copper_polys) = 171 total_poly_num = 171
[+] Check Results (should be all true) res.count(True) = 171  res.count(False) = 0
[+] Check tmp (should be all false) tmp.count(True) = 0  tmp.count(False) = 171
[+] B.dimensions() = (171, 171). LLLing...
[+] Flatter-LLL Done


INFO:logger:start solve_root_jacobian newton


[+] find 87 polynomials with the desired roots in ZZ
[+] the good polynomials indexes : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86]
[+] select first 20 of 171 polynomials to find the roots in ZZ
[+] try the multiple method to find roots


INFO:logger:end solve_root_jacobian newton. elapsed 1.377153


[+] find : roots = [[1892176435689382333859976604, 3566068196028864541485742612, 3808184820143735200706579972, 209311589974575763931512392, 2529144676477570667664998879, 1087461165700887990509367898]]
[+] try the variety method to find roots
[+] try the gcd method to find roots



## ECHNP Solver

The following code shows how to solve the ECHNP problem with the coppersmith's method using the aforementioned lattices.

In [9]:
def echnp_coppersmith_solver(Curve, G, pubA, pubB, kbit, H0, positiveH, negativeH, xQ, d, t=1, 
                             msb=True, lattice="XHS22", first_n=None):
    """solve the ECHNP problem using coppersmith method

    Args:
        Curve (EllipticCurve): the elliptic curve used in the ECDH oracle
        G (point in EllipticCurve): the generator point
        pubA (point in EllipticCurve): the public key of Alice
        pubB (point in EllipticCurve): the public key of Bob
        kbit (int or integer): the number of bits to leak
        H0 (int or integer): the leaked bits of the shared secret of the oracle at 0 i.e. msb leak of ([ab]G + [0]pubA) or ([ab]G + [0]pubB)
        positiveH (list of ints or integers): the leak bits of oracle(m) with positive m = 1,2..., n i.e. msb leak of (abG + [m]pubA) or ([ab]G + [m]pubB)
        negativeH (list of ints or integers): the leak bits of oracle(-m) with negative m = 1,2..., n i.e. msb leak of (abG + [-m]pubA) or ([ab]G + [-m]pubB)
        xQ (list of ints or integers) : the x-coordinates of Q = [m]pubA for m = 1,2,...,n when oracle is based on privA i.e. (abG + [m]pubA) 
                                        the x-coordinates of Q = [m]pubB for m = 1,2,...,n when oracle is based on privB i.e. (abG + [m]pubB)
        d (int or integer): the `d` parameter for constructing the lattice
        t (int or integer): the `t` parameter for constructing the lattice (for XHS22). Default is 1.
        msb (bool, optional): the most significant bit leak or least significant bit leak. Defaults to True i.e MSB leak model.
            **Remarks** if msb is True, the value of H0, positiveH, negativeH should be the msb leak with bit in its original position : (x>> (pbit - kbit)) << (pbit - kbit)
        lattice (string) : the lattice model used : "XHS20" or "XHS22". Default is "XHS22"
    Returns:
        integer or None: the x-coordinate of the shared secret [ab]G if found, None otherwise
    """
    assert len(positiveH) == len(negativeH), "Invalid input"
    assert lattice in ["XHS20", "XHS22"], "unknown lattice"
    R = Curve.base_ring()
    p = R.order()
    pbit = p.nbits()
    ebit = pbit - kbit
    n = len(positiveH)
    CurveParas = (Curve, p, pubA, pubB, G)
    if msb:
        # check form (x>> (pbit - kbit)) << (pbit - kbit)
        if any(h % 2**ebit != 0 for h in [H0] + positiveH + negativeH):
            # transform the msb leak to the form (x>> (pbit - kbit)) << (pbit - kbit)
            H0 = H0 << ebit
            positiveH = [h << ebit for h in positiveH]
            negativeH = [h << ebit for h in negativeH]
    Hs = [ZZ(ph) + ZZ(nh)  for ph, nh in zip(positiveH, negativeH)]
    polys = GetECHNPPolys(H0, Hs, xQ, CurveParas)
    if not msb:
        # lsb case
        x, y = polys[0].parent().gens()
        shift = 2 ** kbit
        # make the leading coeffient of x^2y being zero
        inv_shift = ZZ(inverse_mod(shift ** 3, p))
        polys = [poly(shift *x, shift * y) * inv_shift for poly in polys]
    # generate the lattice polynomials
    if lattice == "XHS22":
        copper_polys = XHS22_Lattice_Polys(n, d, t, p, polys)
    else:
        copper_polys = XHS20_Lattice_Polys(n, d, p, polys)
        
    bounds = [2**ebit] + [2**(ebit+1)] * n
    print(f"[+] Generating {len(copper_polys)} polynomials for coppersmith ({lattice})")
    reduced_polys = coppersmith_multivarivate_reduced_poly(copper_polys, bounds)
    if first_n == None:
        first_n = len(reduced_polys)
    print(f"[+] try several general methods to find roots using {first_n} of {len(reduced_polys)} polynomials")    
    # [TRIANGULATE, GROEBNER, JACOBIAN, HENSEL]
    roots = rootfind_ZZ(reduced_polys[:first_n], bounds)
    if roots is None:
        return None
    for root in roots:
        print(f"[+] find : {root = }")
        # returns only one root
        if type(root) == list:
            e0 = root[0]
        elif type(root) == dict:
            xs = reduced_polys[0].parent().gens()
            e0 = ZZ(root[xs[0]])
        else:
            assert False, "unknown root type"
        if msb:
            return ZZ(H0 + e0)
        else:
            return ZZ(H0 + (e0 << kbit))
    return None

def test_echnp_coppersmith_solver_secp256(kbit, n, d, t=1, lattice="XHS22", msb=True, first_n=None):
    E, G = secp256r1_curve()
    ecdh = ECDH(E, G)
    A, B = ecdh.get_pubs()
    shared_secret = ecdh.get_shared_secret()
    print(f"[+] The shared secret is {shared_secret}")
    p = ZZ(E.base_field().order())
    pbit = p.nbits()
    ebit = pbit - kbit
    print(f"[+] The basic parameters {pbit = } {kbit = } {msb = }")
    print(f"[+] Try to use lattice parameters {n = } {d = } {t = } with {lattice} Lattice")
    CurveParas = (E, p, A, B, G)
    oracle = ecdh.oracle_func(kbit, msb, True)
    h0, hs, xqs, phs, nhs = GenECHNP(oracle, CurveParas, n, kbit, msb)
    result = echnp_coppersmith_solver(E, G, A, B, kbit, h0, phs, nhs, xqs, d, t, msb, lattice, first_n)
    if result is not None:
        print(f"[+] The shared secret : {result = }\n[+] check : {result == shared_secret}")        
    else:
        print("[+] The shared secret is not found")
    print()

In [10]:
# test msb leak
test_echnp_coppersmith_solver_secp256(185, 5, 2, msb=True, first_n = 20)
# test lsb leak
test_echnp_coppersmith_solver_secp256(185, 5, 2, msb=False, first_n = 20)

[+] The shared secret is 31035040855393688989892082732739854146927769970401803013673526706182024193302
[+] The basic parameters pbit = 256 kbit = 185 msb = True
[+] Try to use lattice parameters n = 5 d = 2 t = 1 with XHS22 Lattice
[+] Generating 84 polynomials for coppersmith (XHS22)
[+] B.dimensions() = (84, 84). LLLing...


INFO:logger:start solve_root_jacobian newton


[+] Flatter-LLL Done
[+] try several general methods to find roots using 20 of 84 polynomials


INFO:logger:end solve_root_jacobian newton. elapsed 1.086504


[+] find : root = [138427640008454300950, 1381548549970598773357, 3955787430630802824365, 1552730121401413421387, 3990776596130067070527, 3090437148195427062605]
[+] The shared secret : result = 31035040855393688989892082732739854146927769970401803013673526706182024193302
[+] check : True

[+] The shared secret is 42079639426629304963665128625336271811935347180681192169440599481486389420186
[+] The basic parameters pbit = 256 kbit = 185 msb = False
[+] Try to use lattice parameters n = 5 d = 2 t = 1 with XHS22 Lattice
[+] Generating 84 polynomials for coppersmith (XHS22)
[+] B.dimensions() = (84, 84). LLLing...


INFO:logger:start solve_root_jacobian newton


[+] Flatter-LLL Done
[+] try several general methods to find roots using 20 of 84 polynomials


INFO:logger:end solve_root_jacobian newton. elapsed 4.023036


[+] find : root = [858070184882347107734, 1483037622424439158714, 2742262109445362326256, 2935836563798965211050, 1302113180923331932863, 2328561227447735374936]
[+] The shared secret : result = 42079639426629304963665128625336271811935347180681192169440599481486389420186
[+] check : True



**Another Solver Using the groebner basis**

In [11]:
def echnp_coppersmith_solver_groebner(Curve, G, pubA, pubB, kbit, H0, positiveH, negativeH, xQ, d, t=1, 
                             msb=True, lattice="XHS22", first_n=None):
    """solve the ECHNP problem using coppersmith method

    Args:
        Curve (EllipticCurve): the elliptic curve used in the ECDH oracle
        G (point in EllipticCurve): the generator point
        pubA (point in EllipticCurve): the public key of Alice
        pubB (point in EllipticCurve): the public key of Bob
        kbit (int or integer): the number of bits to leak
        H0 (int or integer): the leaked bits of the shared secret of the oracle at 0 
                            i.e. msb leak of ([ab]G + [0]pubA) or ([ab]G + [0]pubB)
        positiveH (list of ints or integers): the leak bits of oracle(m) with positive m = 1,2..., n 
                            i.e. msb leak of (abG + [m]pubA) or ([ab]G + [m]pubB)
        negativeH (list of ints or integers): the leak bits of oracle(-m) with negative m = 1,2..., n 
                            i.e. msb leak of (abG + [-m]pubA) or ([ab]G + [-m]pubB)
        xQ (list of ints or integers) : the x-coordinates of Q = [m]pubA for m = 1,2,...,n when oracle is based on privA i.e. (abG + [m]pubA) 
                                        the x-coordinates of Q = [m]pubB for m = 1,2,...,n when oracle is based on privB i.e. (abG + [m]pubB)
        d (int or integer): the `d` parameter for constructing the lattice
        t (int or integer): the `t` parameter for constructing the lattice (for XHS22). Default is 1.
        msb (bool, optional): the most significant bit leak or least significant bit leak. Defaults to True i.e MSB leak model.
            **Remarks** if msb is True, the value of H0, positiveH, negativeH should be the msb leak with bit in its original position : (x>> (pbit - kbit)) << (pbit - kbit)
        lattice (string) : the lattice model used : "XHS20" or "XHS22". Default is "XHS22"
        first_n (int or integer) : use the first n polynomials to generate groebner basis. Defaul is all.
    Returns:
        integer or None: the x-coordinate of the shared secret [ab]G if found, None otherwise
    """
    assert len(positiveH) == len(negativeH), "Invalid input"
    assert lattice in ["XHS20", "XHS22"], "unknown lattice"
    R = Curve.base_ring()
    p = R.order()
    pbit = p.nbits()
    ebit = pbit - kbit
    n = len(positiveH)
    CurveParas = (Curve, p, pubA, pubB, G)
    if msb:
        # check form (x>> (pbit - kbit)) << (pbit - kbit)
        if any(h % 2**ebit != 0 for h in [H0] + positiveH + negativeH):
            # transform the msb leak to the form (x>> (pbit - kbit)) << (pbit - kbit)
            H0 = H0 << ebit
            positiveH = [h << ebit for h in positiveH]
            negativeH = [h << ebit for h in negativeH]
    Hs = [ZZ(ph) + ZZ(nh)  for ph, nh in zip(positiveH, negativeH)]
    polys = GetECHNPPolys(H0, Hs, xQ, CurveParas)
    if not msb:
        # lsb case
        x, y = polys[0].parent().gens()
        shift = 2 ** kbit
        # make the leading coeffient of x^2y being zero
        inv_shift = ZZ(inverse_mod(shift ** 3, p))
        polys = [poly(shift *x, shift * y) * inv_shift for poly in polys]
    # generate the lattice polynomials
    if lattice == "XHS22":
        copper_polys = XHS22_Lattice_Polys(n, d, t, p, polys)
    else:
        copper_polys = XHS20_Lattice_Polys(n, d, p, polys)
        
    # ebit = ebit - 1
    bounds = [2**ebit] + [2**(ebit+1)] * n
    print(f"[+] Generating {len(copper_polys)} polynomials for coppersmith ({lattice})")
    reduced_polys = coppersmith_multivarivate_reduced_poly(copper_polys, bounds)
    if first_n == None:
        first_n = len(reduced_polys)
    print(f"[+] try the groebner method to find roots using {first_n} of {len(reduced_polys)} polynomials")
    roots = find_roots_groebner(reduced_polys[0].parent(), reduced_polys[:first_n])
    for root in roots:
        print(f"[+] find : {root = }")
        # returns only one root
        if type(root) == list:
            e0 = root[0]
        elif type(root) == dict:
            xs = reduced_polys[0].parent().gens()
            e0 = ZZ(root[xs[0]])
        else:
            assert False, "unknown root type"
        if msb:
            return ZZ(H0 + e0)
        else:
            return ZZ(H0 + (e0 << kbit))
    return None

def test_echnp_coppersmith_solver_groebner_secp256(kbit, n, d, t=1, lattice="XHS22", msb=True, first_n=None):
    E, G = secp256r1_curve()
    ecdh = ECDH(E, G)
    A, B = ecdh.get_pubs()
    shared_secret = ecdh.get_shared_secret()
    print(f"[+] The shared secret is {shared_secret}")
    p = ZZ(E.base_field().order())
    pbit = p.nbits()
    ebit = pbit - kbit
    print(f"[+] The basic parameters {pbit = } {kbit = } {msb = }")
    print(f"[+] Try to use lattice parameters {n = } {d = } {t = } with {lattice} Lattice")
    CurveParas = (E, p, A, B, G)
    oracle = ecdh.oracle_func(kbit, msb, True)
    h0, hs, xqs, phs, nhs = GenECHNP(oracle, CurveParas, n, kbit, msb)
    result = echnp_coppersmith_solver_groebner(E, G, A, B, kbit, h0, phs, nhs, xqs, 
                                               d, t, msb, lattice, first_n)
    if result is not None:
        print(f"[+] The shared secret : {result = }\n[+] check : {result == shared_secret}")
    else:
        print("[+] The shared secret is not found")

In [12]:
# msb
test_echnp_coppersmith_solver_groebner_secp256(kbit = 180, msb = True, n = 4, d = 2, t = 1, first_n = 80)
# lsb
test_echnp_coppersmith_solver_groebner_secp256(kbit = 180, msb = False, n = 4, d = 2, t = 1, first_n = 80)

[+] The shared secret is 109904823151624688788482464102463898517023245988688210206461088288264367966117
[+] The basic parameters pbit = 256 kbit = 180 msb = True
[+] Try to use lattice parameters n = 4 d = 2 t = 1 with XHS22 Lattice
[+] Generating 52 polynomials for coppersmith (XHS22)
[+] B.dimensions() = (52, 52). LLLing...
[+] Flatter-LLL Done
[+] try the groebner method to find roots using 80 of 52 polynomials
[+] find : root = {x0: 37207786966393962956709, y1: 113756542916188262439715, y2: 127994346698741634883002, y3: 54254022680026845790223, y4: 87533637174673223190910}
[+] The shared secret : result = 109904823151624688788482464102463898517023245988688210206461088288264367966117
[+] check : True
[+] The shared secret is 107660455571394373108526502451975528373280533202980229112330594015871759530643
[+] The basic parameters pbit = 256 kbit = 180 msb = False
[+] Try to use lattice parameters n = 4 d = 2 t = 1 with XHS22 Lattice
[+] Generating 52 polynomials for coppersmith (XHS22)

## Optimized Solver by Recentering

The optimized solver with the same bound described in original paper using recentering method.

In [13]:
def echnp_coppersmith_solver_groebner_op(Curve, G, pubA, pubB, kbit, H0, positiveH, negativeH, xQ, d, t=1, 
                             msb=True, lattice="XHS22", first_n=None):
    """solve the ECHNP problem using coppersmith method

    Args:
        Curve (EllipticCurve): the elliptic curve used in the ECDH oracle
        G (point in EllipticCurve): the generator point
        pubA (point in EllipticCurve): the public key of Alice
        pubB (point in EllipticCurve): the public key of Bob
        kbit (int or integer): the number of bits to leak
        H0 (int or integer): the leaked bits of the shared secret of the oracle at 0 
                            i.e. msb leak of ([ab]G + [0]pubA) or ([ab]G + [0]pubB)
        positiveH (list of ints or integers): the leak bits of oracle(m) with positive m = 1,2..., n 
                            i.e. msb leak of (abG + [m]pubA) or ([ab]G + [m]pubB)
        negativeH (list of ints or integers): the leak bits of oracle(-m) with negative m = 1,2..., n 
                            i.e. msb leak of (abG + [-m]pubA) or ([ab]G + [-m]pubB)
        xQ (list of ints or integers) : the x-coordinates of Q = [m]pubA for m = 1,2,...,n when oracle is based on privA i.e. (abG + [m]pubA) 
                                        the x-coordinates of Q = [m]pubB for m = 1,2,...,n when oracle is based on privB i.e. (abG + [m]pubB)
        d (int or integer): the `d` parameter for constructing the lattice
        t (int or integer): the `t` parameter for constructing the lattice (for XHS22). Default is 1.
        msb (bool, optional): the most significant bit leak or least significant bit leak. Defaults to True i.e MSB leak model.
            **Remarks** if msb is True, the value of H0, positiveH, negativeH should be the msb leak with bit in its original position : (x>> (pbit - kbit)) << (pbit - kbit)
        lattice (string) : the lattice model used : "XHS20" or "XHS22". Default is "XHS22"
        first_n (int or integer) : use the first n polynomials to generate groebner basis. Defaul is all.
    Returns:
        integer or None: the x-coordinate of the shared secret [ab]G if found, None otherwise
    """
    assert len(positiveH) == len(negativeH), "Invalid input"
    assert lattice in ["XHS20", "XHS22"], "unknown lattice"
    R = Curve.base_ring()
    p = R.order()
    pbit = p.nbits()
    ebit = pbit - kbit
    n = len(positiveH)
    CurveParas = (Curve, p, pubA, pubB, G)
    if msb:
        # check form (x>> (pbit - kbit)) << (pbit - kbit)
        if any(h % 2**ebit != 0 for h in [H0] + positiveH + negativeH):
            # transform the msb leak to the form (x>> (pbit - kbit)) << (pbit - kbit)
            H0 = H0 << ebit
            positiveH = [h << ebit for h in positiveH]
            negativeH = [h << ebit for h in negativeH]
    Hs = [ZZ(ph) + ZZ(nh)  for ph, nh in zip(positiveH, negativeH)]
    polys = GetECHNPPolys(H0, Hs, xQ, CurveParas)
    if not msb:
        # lsb case
        x, y = polys[0].parent().gens()
        shift = 2 ** kbit
        # make the leading coeffient of x^2y being zero
        inv_shift = ZZ(inverse_mod(shift ** 3, p))
        polys = [poly(shift *x, shift * y) * inv_shift for poly in polys]
        
    # recentering optimization  
    ebit = ebit - 1
    bounds = [2**ebit] + [2**(ebit+1)] * n
    new_poly = []
    for poly in polys:
        pr = poly.parent()
        xs = list(pr.gens())
        shift_xs = {x: x + b for x,b in zip(xs, bounds)}
        new_poly.append(pr(poly.subs(shift_xs)))
    polys = new_poly[:]
    
    # generate the lattice polynomials
    if lattice == "XHS22":
        copper_polys = XHS22_Lattice_Polys(n, d, t, p, polys)
    else:
        copper_polys = XHS20_Lattice_Polys(n, d, p, polys)
    
    print(f"[+] Generating {len(copper_polys)} polynomials for coppersmith ({lattice})")
    reduced_polys = coppersmith_multivarivate_reduced_poly(copper_polys, bounds)
    if first_n == None:
        first_n = len(reduced_polys)
    print(f"[+] try the groebner method to find roots using {first_n} of {len(reduced_polys)} polynomials")
    roots = find_roots_groebner(reduced_polys[0].parent(), reduced_polys[:first_n])
    for root in roots:
        print(f"[+] find : {root = }")
        # returns only one root
        if type(root) == list:
            e0 = root[0] + 2**ebit
        elif type(root) == dict:
            xs = reduced_polys[0].parent().gens()
            e0 = ZZ(root[xs[0]]) + 2**ebit
        else:
            assert False, "unknown root type"
        if msb:
            return ZZ(H0 + e0)
        else:
            return ZZ(H0 + (e0 << kbit))
    return None

def test_echnp_coppersmith_solver_groebner_secp256_op(kbit, n, d, t=1, lattice="XHS22", msb=True, first_n=None):
    E, G = secp256r1_curve()
    ecdh = ECDH(E, G)
    A, B = ecdh.get_pubs()
    shared_secret = ecdh.get_shared_secret()
    print(f"[+] The shared secret is {shared_secret}")
    p = ZZ(E.base_field().order())
    pbit = p.nbits()
    ebit = pbit - kbit
    print(f"[+] The basic parameters {pbit = } {kbit = } {msb = }")
    print(f"[+] Try to use lattice parameters {n = } {d = } {t = } with {lattice} Lattice")
    CurveParas = (E, p, A, B, G)
    oracle = ecdh.oracle_func(kbit, msb, True)
    h0, hs, xqs, phs, nhs = GenECHNP(oracle, CurveParas, n, kbit, msb)
    result = echnp_coppersmith_solver_groebner_op(E, G, A, B, kbit, h0, phs, nhs, xqs, 
                                               d, t, msb, lattice, first_n)
    if result is not None:
        print(f"[+] The shared secret : {result = }\n[+] check : {result == shared_secret}")
    else:
        print("[+] The shared secret is not found")
    print()
    
def echnp_coppersmith_solver_optimized(Curve, G, pubA, pubB, kbit, H0, positiveH, negativeH, xQ, d, t=1, 
                             msb=True, lattice="XHS22", first_n=None):
    """solve the ECHNP problem using coppersmith method with recentering optimization

    Args:
        Curve (EllipticCurve): the elliptic curve used in the ECDH oracle
        G (point in EllipticCurve): the generator point
        pubA (point in EllipticCurve): the public key of Alice
        pubB (point in EllipticCurve): the public key of Bob
        kbit (int or integer): the number of bits to leak
        H0 (int or integer): the leaked bits of the shared secret of the oracle at 0 i.e. msb leak of ([ab]G + [0]pubA) or ([ab]G + [0]pubB)
        positiveH (list of ints or integers): the leak bits of oracle(m) with positive m = 1,2..., n i.e. msb leak of (abG + [m]pubA) or ([ab]G + [m]pubB)
        negativeH (list of ints or integers): the leak bits of oracle(-m) with negative m = 1,2..., n i.e. msb leak of (abG + [-m]pubA) or ([ab]G + [-m]pubB)
        xQ (list of ints or integers) : the x-coordinates of Q = [m]pubA for m = 1,2,...,n when oracle is based on privA i.e. (abG + [m]pubA) 
                                        the x-coordinates of Q = [m]pubB for m = 1,2,...,n when oracle is based on privB i.e. (abG + [m]pubB)
        d (int or integer): the `d` parameter for constructing the lattice
        t (int or integer): the `t` parameter for constructing the lattice (for XHS22). Default is 1.
        msb (bool, optional): the most significant bit leak or least significant bit leak. Defaults to True i.e MSB leak model.
            **Remarks** if msb is True, the value of H0, positiveH, negativeH should be the msb leak with bit in its original position : (x>> (pbit - kbit)) << (pbit - kbit)
        lattice (string) : the lattice model used : "XHS20" or "XHS22". Default is "XHS22"
        first_n (int or integer) : use the first n polynomials to generate groebner basis. Defaul is all.
    Returns:
        integer or None: the x-coordinate of the shared secret [ab]G if found, None otherwise
    """
    assert len(positiveH) == len(negativeH), "Invalid input"
    assert lattice in ["XHS20", "XHS22"], "unknown lattice"
    R = Curve.base_ring()
    p = R.order()
    pbit = p.nbits()
    ebit = pbit - kbit
    n = len(positiveH)
    CurveParas = (Curve, p, pubA, pubB, G)
    if msb:
        # check form (x>> (pbit - kbit)) << (pbit - kbit)
        if any(h % 2**ebit != 0 for h in [H0] + positiveH + negativeH):
            # transform the msb leak to the form (x>> (pbit - kbit)) << (pbit - kbit)
            H0 = H0 << ebit
            positiveH = [h << ebit for h in positiveH]
            negativeH = [h << ebit for h in negativeH]
    Hs = [ZZ(ph) + ZZ(nh)  for ph, nh in zip(positiveH, negativeH)]
    polys = GetECHNPPolys(H0, Hs, xQ, CurveParas)
    if not msb:
        # lsb case
        x, y = polys[0].parent().gens()
        shift = 2 ** kbit
        # make the leading coeffient of x^2y being zero
        inv_shift = ZZ(inverse_mod(shift ** 3, p))
        polys = [poly(shift *x, shift * y) * inv_shift for poly in polys]
      
    # recentering optimization  
    ebit = ebit - 1
    bounds = [2**ebit] + [2**(ebit+1)] * n
    new_poly = []
    for poly in polys:
        pr = poly.parent()
        xs = list(pr.gens())
        shift_xs = {x: x + b for x,b in zip(xs, bounds)}
        new_poly.append(pr(poly.subs(shift_xs)))
    polys = new_poly[:]
    
    # generate the lattice polynomials
    if lattice == "XHS22":
        copper_polys = XHS22_Lattice_Polys(n, d, t, p, polys)
    else:
        copper_polys = XHS20_Lattice_Polys(n, d, p, polys)
    
    print(f"[+] Generating {len(copper_polys)} polynomials for coppersmith ({lattice})")
    reduced_polys = coppersmith_multivarivate_reduced_poly(copper_polys, bounds)
    # [TRIANGULATE, GROEBNER, JACOBIAN, HENSEL]
    if first_n == None:
        first_n = len(reduced_polys)
    print(f"[+] try the several methods to find roots using {first_n} of {len(reduced_polys)} polynomials")
    roots = rootfind_ZZ(reduced_polys[:first_n], bounds)
    if roots is None:
        return None
    for root in roots:
        print(f"[+] find : {root = }")
        # returns only one root
        if type(root) == list:
            e0 = ZZ(root[0]) + 2**ebit
        elif type(root) == dict:
            xs = reduced_polys[0].parent().gens()
            e0 = ZZ(root[xs[0]]) + 2**ebit
        else:
            assert False, "unknown root type"
        if msb:
            return ZZ(H0 + e0)
        else:
            return ZZ(H0 + (e0 << kbit))
    return None

def test_echnp_coppersmith_optimized_solver_secp256(kbit, n, d, t=1, lattice="XHS22", msb=True, first_n=None):
    E, G = secp256r1_curve()
    ecdh = ECDH(E, G)
    A, B = ecdh.get_pubs()
    shared_secret = ecdh.get_shared_secret()
    print(f"[+] The shared secret is {shared_secret}")
    p = ZZ(E.base_field().order())
    pbit = p.nbits()
    ebit = pbit - kbit
    print(f"[+] The basic parameters {pbit = } {kbit = } {msb = }")
    print(f"[+] Try to use lattice parameters {n = } {d = } {t = } with {lattice} Lattice")
    CurveParas = (E, p, A, B, G)
    oracle = ecdh.oracle_func(kbit, msb, True)
    h0, hs, xqs, phs, nhs = GenECHNP(oracle, CurveParas, n, kbit, msb)
    result = echnp_coppersmith_solver_optimized(E, G, A, B, kbit, h0, phs, nhs, xqs, 
                                                d, t, msb, lattice, first_n)
    if result is not None:
        print(f"[+] The shared secret : {result = }\n[+] check : {result == shared_secret}")        
    else:
        print("[+] The shared secret is not found")
    print()
        
def test_Coppersmallroot_With_Details_Secp256_XHS22_op(kbit = 172, n = 5, d = 2, t = 1, first_n = 10):
    E, G = secp256r1_curve()
    p = ZZ(E.base_field().order())
    pbit = p.nbits()
    ebit = pbit - kbit

    msb = True
    print(f"[+] The basic parameters {pbit = } {kbit = } {msb = }")
    print(f"[+] Try to use lattice parameters {n = } {d = } {t = } with XHS22 Lattice")
    polys, e, es = ECHNP_Polys(kbit, n)
    roots = [e] + es
    
    ebit = ebit - 1
    bounds = [2**ebit] + [2**(ebit+1)] * n
    new_poly = []
    for poly in polys:
        pr = poly.parent()
        xs = list(pr.gens())
        shift_xs = {x: x + b for x,b in zip(xs, bounds)}
        new_poly.append(pr(poly.subs(shift_xs)))
    # new roots
    for i, poly in enumerate(polys):
        assert poly(roots[0], roots[i+1]) % p == 0,  "Invalid polynomial for the original roots"
        
    roots = [root - bound for root, bound in zip(roots, bounds)]
    polys = new_poly[:]
    for i, poly in enumerate(polys):
        assert poly(roots[0], roots[i+1]) % p == 0,  "Invalid polynomial for the new roots"
    
    count_poly = lambda n, d, t : (2*d) * sum(binomial(n, l) for l in range(0,d+1)) + (t + 1) * binomial(n, d + 1)
    copper_polys = XHS22_Lattice_Polys(n, d, t, p, polys)
    total_poly_num = count_poly(n, d, t)
    print(f"[+] Shifted Polynomials test passed {len(copper_polys) = } {total_poly_num = }")
    res = []    
    tmp = []
    useful_copper_polys = []
    for poly in copper_polys:
        if poly == 0:
            continue
        useful_copper_polys.append(poly)
        res.append(poly(*roots) % p**d == 0)       
        tmp.append(poly(*roots) % p**(d+1) == 0)
        
    print(f"[+] Check Results (should be all true) {res.count(True) = } { res.count(False) = }")
    print(f"[+] Check tmp (should be all false) {tmp.count(True) = } { tmp.count(False) = }")
    polys = coppersmith_multivarivate_reduced_poly(useful_copper_polys, bounds)
    # check these polys
    good_polys_idx = []
    for i, poly in enumerate(polys):
        poly = poly.change_ring(ZZ)
        assert poly(*roots) % p**d == 0, "not copper poly"
        # check wether the poly has the original roots in ZZ
        if poly(*roots) == 0 and poly % p**d != 0:
            good_polys_idx.append(i)
    print(f"[+] find {len(good_polys_idx)} polynomials with the desired roots in ZZ")
    print(f"[+] the good polynomials indexes : {good_polys_idx}")
    if first_n is None:
        first_n = len(polys)
    final_polys = polys[:first_n]
        
    print(f"[+] select first {first_n} of {len(polys)} polynomials to find the roots in ZZ")
    # print("[+] try the groebner method to find roots")
    # roots = find_roots_groebner(polys[0].parent(), final_polys)
    # for root in roots:        
    #     print(f"[+] find : {root = }")
        
    print("[+] try the multiple method to find roots")
    roots = rootfind_ZZ(final_polys, bounds)
    print(f"[+] find : {roots = }")
    
    print("[+] try the variety method to find roots")
    roots = find_roots_variety(polys[0].parent(), final_polys)
    for root in roots:
        print(f"[+] find : {root = }")
        
    print("[+] try the gcd method to find roots")
    roots = find_roots_gcd(polys[0].parent(), final_polys)
    for root in roots:
        print(f"[+] find : {root = }")
    print()
        
if __name__ == "__main__":
    test_Coppersmallroot_With_Details_Secp256_XHS22_op(first_n = 40)
    # msb
    test_echnp_coppersmith_solver_groebner_secp256_op(kbit = 164, msb = True, n = 5, d = 3, t = 2, first_n = 80)
    # lsb
    test_echnp_coppersmith_solver_groebner_secp256_op(kbit = 164, msb = False, n = 5, d = 3, t = 2, first_n = 80)
    
    # test msb leak
    test_echnp_coppersmith_optimized_solver_secp256(kbit = 164, msb = True, n = 5, d = 3, t = 2, first_n = 20)
    # test lsb leak
    test_echnp_coppersmith_optimized_solver_secp256(kbit = 164, msb = True, n = 5, d = 3, t = 2, first_n = 20)

[+] The basic parameters pbit = 256 kbit = 172 msb = True
[+] Try to use lattice parameters n = 5 d = 2 t = 1 with XHS22 Lattice
[+] Shifted Polynomials test passed len(copper_polys) = 84 total_poly_num = 84
[+] Check Results (should be all true) res.count(True) = 84  res.count(False) = 0
[+] Check tmp (should be all false) tmp.count(True) = 0  tmp.count(False) = 84
[+] B.dimensions() = (84, 84). LLLing...


INFO:logger:start solve_root_jacobian newton


[+] Flatter-LLL Done
[+] find 83 polynomials with the desired roots in ZZ
[+] the good polynomials indexes : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]
[+] select first 40 of 84 polynomials to find the roots in ZZ
[+] try the multiple method to find roots


INFO:logger:end solve_root_jacobian newton. elapsed 4.017351


[+] find : roots = [[6571968945535555570817796, -9983948573027934851165635, -12217790441711508073956541, -2507643748278388014089846, 3868766907851350791303223, 5191389753415246648899020]]
[+] try the variety method to find roots
[+] try the gcd method to find roots

[+] The shared secret is 2612144256024688237143762885995309127009495150008089623055386970663035669085
[+] The basic parameters pbit = 256 kbit = 164 msb = True
[+] Try to use lattice parameters n = 5 d = 3 t = 2 with XHS22 Lattice
[+] Generating 171 polynomials for coppersmith (XHS22)
[+] B.dimensions() = (171, 171). LLLing...
[+] Flatter-LLL Done
[+] try the groebner method to find roots using 80 of 171 polynomials
[+] find : root = {x0: 2279368124000724752681994845, y1: -349086628473733120615741019, y2: 3252545445841213482329437777, y3: 3549013605846289644604551210, y4: 371131644603220196709659150, y5: 29757007297586620303350993}
[+] The shared secret : result = 261214425602468823714376288599530912700949515000808962305538

...

[+] find : root = [-1587941370110245502961469151, 518749998614478353016559060, -1390931177728929462518337328, -4185669707115490779170527750, -216985065707944192116219104, -1091140248246503403462857230]
[+] The shared secret : result = 13410301625082747820546151041404445539297150040316608870447595473497285835041
[+] check : True

[+] The shared secret is 93391653508329261507318513482266927939321386608054709269728660650503406502388
[+] The basic parameters pbit = 256 kbit = 164 msb = True
[+] Try to use lattice parameters n = 5 d = 3 t = 2 with XHS22 Lattice
[+] Generating 171 polynomials for coppersmith (XHS22)
[+] B.dimensions() = (171, 171). LLLing...


INFO:logger:start solve_root_jacobian newton


[+] Flatter-LLL Done
[+] try the several methods to find roots using 20 of 171 polynomials


INFO:logger:end solve_root_jacobian newton. elapsed 1.279029


[+] find : root = [-610187675938228277166325260, -3014453454146029715877822846, 2210718752196346832193978310, -2545654420620274546666066513, -447315142210334174425501683, -3041252866072064888528221485]
[+] The shared secret : result = 93391653508329261507318513482266927939321386608054709269728660650503406502388
[+] check : True

